# Test of HiPace300 Pumps on Transfer and Depo Chamber

## Equipment Configuration:
- **HiPace300 Pumps**: Connected to Transfer and Deposition Chambers
- **HiScroll12**: One unit connected as backing pump
- **Pressure Sensors**: Two total sensors connected to both Omnicontrol 300 units

## Test Overview:
This notebook contains tests and monitoring for the HiPace300 turbo molecular pump system with integrated pressure monitoring and control capabilities.

## 1. Import Required Libraries and Setup


In [ ]:
import sys
import threading
import time
from datetime import datetime
import asyncio
from typing import Dict, Optional, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle

from loguru import logger
import os
from pathlib import Path

# Add path to src modules
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

# Import current device modules
from devices.pfeiffer.hiscroll12.hiscroll12 import HiScroll12
from devices.pfeiffer.hipacebus import HiPace300Bus

## 2. Setup Shared Logging System

In [ ]:
# Get the repository root and create shared logs directory
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Create shared log file for all devices
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
shared_log_file = log_dir / f"pump_locker_test_{timestamp}.log"

# Configure shared logger
logger.remove()  # Remove default logger

# Add console logger with INFO level
logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")

# Add shared file logger with DEBUG level
logger.add(
    str(shared_log_file),
    level="DEBUG",
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {name}:{function}:{line} | {message}",
    rotation="1 day",
    retention="30 days",
    compression="zip"
)

logger.info("Pump Locker Test system initialized with shared logging")
print(f"Repository root: {repo_root}")
print(f"Shared logs will be saved to: {shared_log_file}")

# 3. Set COM-Ports & Adresses

In [ ]:
# Device configuration dictionaries
# Modify these COM ports and addresses according to your actual hardware setup

HiScroll12 = {
    'com_port': 'COM19',
    'device_address': 2,
    'device_type': 'HiScroll12',
    'description': 'Backing pump for HiPace300 systems'
}

HiPace300_Transfer = {
    'com_port': 'COM20',
    'device_address': 1,
    'device_type': 'HiPace300',
    'description': 'Transfer chamber turbo pump'
}

HiPace300_Depo = {
    'com_port': 'COM21',
    'device_address': 1,
    'device_type': 'HiPace300', 
    'description': 'Deposition chamber turbo pump'
}

# Print configuration summary
print("Device Configuration Summary:")
print("=" * 40)
print(f"HiScroll12 (Backing pump):")
print(f"  COM Port: {HiScroll12['com_port']}")
print(f"  Address: {HiScroll12['device_address']}")
print(f"  Description: {HiScroll12['description']}")

print(f"\nHiPace300 Transfer:")
print(f"  COM Port: {HiPace300_Transfer['com_port']}")
print(f"  Address: {HiPace300_Transfer['device_address']}")
print(f"  Description: {HiPace300_Transfer['description']}")

print(f"\nHiPace300 Depo:")
print(f"  COM Port: {HiPace300_Depo['com_port']}")
print(f"  Address: {HiPace300_Depo['device_address']}")
print(f"  Description: {HiPace300_Depo['description']}")

print("\n✅ Device dictionaries configured successfully")

# 4. Connect Devices

In [ ]:
# Initialize and connect HiScroll12
hiscroll = HiScroll12("hiscroll12", HiScroll12['com_port'], HiScroll12['device_address'], logger)
hiscroll.connect()
print(f"✅ HiScroll12 connected on {HiScroll12['com_port']}")

In [ ]:
# Initialize and connect HiPace300 Transfer
hipace_transfer = HiPace300Bus("hipace_transfer", HiPace300_Transfer['com_port'], HiPace300_Transfer['device_address'], logger)
hipace_transfer.connect()
print(f"✅ HiPace300 Transfer connected on {HiPace300_Transfer['com_port']}")

In [ ]:
# Initialize and connect HiPace300 Depo
hipace_depo = HiPace300Bus("hipace_depo", HiPace300_Depo['com_port'], HiPace300_Depo['device_address'], logger)
hipace_depo.connect()
print(f"✅ HiPace300 Depo connected on {HiPace300_Depo['com_port']}")

# 5. Debugging/Testing Connection manually